# Reproduce LineVul

In [1]:
from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig

In [2]:
# there are some warning from transformer
# due to its verbose, disable

from transformers import logging
logging.set_verbosity(40)

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler

In [4]:
import numpy as np

In [5]:
import pandas as pd
from tqdm.autonotebook import tqdm

In [6]:
from linevul_model import Model
from linevul_helpers import TextDataset
from linevul_extra import extract_line_attention, linevul_predict

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [8]:
config = RobertaConfig.from_pretrained('microsoft/codebert-base')
config.num_labels = 1
config.num_attention_heads = 12

In [9]:
# get from LineVul
checkpoint = '/home/hqn650/LineVul/linevul/saved_models/checkpoint-best-f1/12heads_linevul_model.bin'

In [10]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

In [11]:
pre_train = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', 
                                                             config=config, 
                                                             ignore_mismatched_sizes=True)

In [12]:
from dataclasses import dataclass

@dataclass
class Args:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_gpu = torch.cuda.device_count()
    use_non_pretrained_model = False
    block_size = 512
    test_data_file = '/home/hqn650/LineVul/data/big-vul_dataset/test.csv'
    code_length=256
    do_local_explanation=True
    reasoning_method='attention'
    seed=42
    num_attention_heads=12
    do_sorting_by_line_scores=False
    do_sorting_by_pred_prob=False
    top_k_constant=10
    use_word_level_tokenizer=False
    eval_batch_size=512
    
args = Args()

In [13]:
model = Model(pre_train, config, tokenizer, args)

In [14]:
model.load_state_dict(torch.load(checkpoint, map_location=args.device))
model.to(args.device)

Model(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [15]:
test_dataset = TextDataset(tokenizer, args, file_type='test')

  0%|          | 0/18864 [00:00<?, ?it/s]

In [16]:
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args.eval_batch_size, num_workers=0)

In [17]:
# multi-gpu evaluate
if args.n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [18]:
result, y_trues, y_preds = linevul_predict(model, test_dataloader, args.device)

/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [19]:
result

{'test_accuracy': 0.9909351145038168,
 'test_recall': 0.8635071090047394,
 'test_precision': 0.9712153518123667,
 'test_f1': 0.9141996989463121,
 'test_threshold': 0.5}

In [20]:
correct_indices = np.where((y_trues == y_preds))
correct_indices = list(correct_indices[0])

In [21]:
tp_indices = np.where((y_trues == y_preds) & (y_trues == 1))
tp_indices = list(tp_indices[0])

In [22]:
# after identify true positive sample, create new loader for explaination

dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

In [23]:
df = pd.read_csv(args.test_data_file)

In [24]:
top_k_constant = [args.top_k_constant]

In [45]:
model.eval()
index = 0
progress_bar = tqdm(dataloader, total=len(dataloader))
extract_list = []
for mini_batch in progress_bar:
    if index in tp_indices:
        (input_ids, labels) = mini_batch
        ids = input_ids[0].detach().tolist()
        all_tokens = tokenizer.convert_ids_to_tokens(ids)
        all_tokens = [token.replace("Ġ", "") for token in all_tokens]
        all_tokens = [token.replace("ĉ", "Ċ") for token in all_tokens]
        with torch.no_grad():
            prob, attentions = model(input_ids=input_ids, output_attentions=True)
        lines_with_score, n_lines = extract_line_attention(attentions, all_tokens)
        extract_list.append((index, lines_with_score, n_lines))
    index += 1

  0%|          | 0/18864 [00:00<?, ?it/s]

In [46]:
extract_list[-1]

(18822,
 [(2,
   'command_line->AppendSwitch(switches::kEnableThreadedCompositing);',
   1153.4075927734375),
  (1, 'GpuFeatureTest::SetUpCommandLine(command_line);', 963.8869018554688),
  (0,
   '<s>virtualvoidSetUpCommandLine(CommandLine*command_line){',
   850.890869140625),
  (3, '}', 151.55853271484375)],
 4)

In [32]:
lines_with_score

[(2,
  'command_line->AppendSwitch(switches::kEnableThreadedCompositing);',
  1153.4075927734375),
 (1, 'GpuFeatureTest::SetUpCommandLine(command_line);', 963.8869018554688),
 (0,
  '<s>virtualvoidSetUpCommandLine(CommandLine*command_line){',
  850.890869140625),
 (3, '}', 151.55853271484375)]

In [42]:
index

18864

In [48]:
sorted_lines = sorted(lines_with_score, key=lambda x: x[2], reverse=True)
sorted_lines[:max(0.1 * n_lines, 1)]

[(2,
  'command_line->AppendSwitch(switches::kEnableThreadedCompositing);',
  1153.4075927734375)]

In [49]:
import codecs

context = df.iloc[18822]['func_before']
modified_context = codecs.decode(context, 'unicode_escape')

new_variable = modified_context.replace(r'\n', '\n')

print(new_variable)


  virtual void SetUpCommandLine(CommandLine* command_line) {
    GpuFeatureTest::SetUpCommandLine(command_line);
    command_line->AppendSwitch(switches::kEnableThreadedCompositing);
  }

